# Row-Level Filtering with UDFs on Materialized Views

## Overview
This notebook demonstrates how to implement **row-level filtering** on materialized views using a filtering UDF. Unlike column masking, this approach filters entire rows based on whether the current user is in an `access_list` column.

## Use Case
Create a materialized view where:
* Each row has an `access_list` array containing authorized user emails
* A UDF checks if `current_user()` is in the access list
* Rows are filtered in the WHERE clause based on access
* Different users see different subsets of data

## Test Scenarios
We'll create 4 types of rows:
1. Only `jwneil17@gmail.com` has access
2. Only `john.neil@databricks.com` has access
3. Both users have access
4. Neither user has access (row hidden from both)

## Step 1: Create Access Check UDF

The UDF checks if the current user's email is in the provided access list array:
* Returns **TRUE** if user is in the list
* Returns **FALSE** if user is not in the list

In [0]:
-- Create a UDF that checks if current user is in the access list
CREATE OR REPLACE FUNCTION main.default.has_access(access_list ARRAY<STRING>)
RETURNS BOOLEAN
RETURN array_contains(access_list, current_user());

In [0]:
select
has_access(array('john.neil@databricks.com'))

## Step 2: Create Base Table with Access Lists

We'll create a base table that adds an `access_list` column to sample taxi trip data with 4 different access scenarios.

In [0]:
-- Create a base table with access_list column
CREATE OR REPLACE TABLE main.default.nyctaxi_trips_with_access AS
SELECT 
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  trip_distance,
  fare_amount,
  pickup_zip,
  dropoff_zip,
  CASE 
    -- Scenario 1: Only jwneil17@gmail.com has access (25% of rows)
    WHEN MOD(CAST(unix_timestamp(tpep_pickup_datetime) AS BIGINT), 4) = 0 
      THEN array('jwneil17@gmail.com')
    -- Scenario 2: Only john.neil@databricks.com has access (25% of rows)
    WHEN MOD(CAST(unix_timestamp(tpep_pickup_datetime) AS BIGINT), 4) = 1 
      THEN array('john.neil@databricks.com')
    -- Scenario 3: Both users have access (25% of rows)
    WHEN MOD(CAST(unix_timestamp(tpep_pickup_datetime) AS BIGINT), 4) = 2 
      THEN array('jwneil17@gmail.com', 'john.neil@databricks.com')
    -- Scenario 4: Neither user has access (25% of rows)
    ELSE array('other.user@example.com')
  END AS access_list
FROM samples.nyctaxi.trips
LIMIT 1000;

In [0]:
-- Verify the access list distribution
SELECT 
  CASE 
    WHEN array_contains(access_list, 'jwneil17@gmail.com') AND array_contains(access_list, 'john.neil@databricks.com') THEN 'Both users'
    WHEN array_contains(access_list, 'jwneil17@gmail.com') THEN 'Only jwneil17@gmail.com'
    WHEN array_contains(access_list, 'john.neil@databricks.com') THEN 'Only john.neil@databricks.com'
    ELSE 'Neither user'
  END AS access_scenario,
  COUNT(*) as row_count
FROM main.default.nyctaxi_trips_with_access
GROUP BY access_scenario
ORDER BY access_scenario;

## Step 3: Create Materialized View with Row Filtering

The materialized view uses the `has_access()` UDF in the WHERE clause to filter rows based on the current user's access.

In [0]:
-- Create materialized view with row-level filtering using ROW FILTER clause
CREATE MATERIALIZED VIEW main.default.nyctaxi_trips_filtered
WITH ROW FILTER main.default.has_access ON (access_list)
AS
SELECT 
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  trip_distance,
  fare_amount,
  pickup_zip,
  dropoff_zip,
  access_list
FROM main.default.nyctaxi_trips_with_access;

In [0]:
select
*
from main.default.nyctaxi_trips_filtered
where pickup_zip = '10001'
order by access_list

## Step 4: Grant Permissions

Grant necessary permissions to the other user to query the materialized view.

In [0]:
-- Grant permissions to john.neil@databricks.com
GRANT USAGE ON CATALOG main TO `john.neil@databricks.com`;
GRANT USAGE ON SCHEMA main.default TO `john.neil@databricks.com`;
GRANT SELECT ON MATERIALIZED VIEW main.default.nyctaxi_trips_filtered TO `john.neil@databricks.com`;
GRANT SELECT ON TABLE main.default.nyctaxi_trips_with_access TO `john.neil@databricks.com`;

## Step 5: Test the Materialized View

Query the materialized view to see which rows are visible to the current user.

In [0]:
-- Test the filtered materialized view
SELECT 
  current_user() as querying_user,
  access_list,
  tpep_pickup_datetime,
  trip_distance,
  fare_amount
FROM main.default.nyctaxi_trips_filtered
LIMIT 20;

In [0]:
-- Count how many rows are visible to current user
SELECT 
  current_user() as querying_user,
  COUNT(*) as visible_rows
FROM main.default.nyctaxi_trips_filtered;

## Results

**When queried by `jwneil17@gmail.com`:**
* Sees rows where access_list contains `jwneil17@gmail.com`
* Should see ~50% of total rows (scenarios 1 and 3)

**When queried by `john.neil@databricks.com`:**
* Sees rows where access_list contains `john.neil@databricks.com`
* Should see ~50% of total rows (scenarios 2 and 3)

**Rows neither user sees:**
* ~25% of rows with `other.user@example.com` in access_list

---

## Key Takeaways

* **Row-level filtering** on materialized views is achieved using the `WITH ROW FILTER` clause
* **Syntax**: `WITH ROW FILTER function_name ON (column_name)` applies the UDF automatically
* **Access lists** stored as arrays provide flexible multi-user access control
* **`array_contains()`** combined with `current_user()` enables dynamic filtering
* **No data duplication** - security enforced at query time
* **Performance** - materialized views still provide query optimization benefits
* **Automatic enforcement** - the filter is applied transparently without WHERE clauses in queries